# Prerequisites

In [0]:
# Update packages and install required java version
!apt-get update
!apt-get install openjdk-21-jdk-headless -qq > /dev/null

# download and unzip spark
!wget -nc -q https://downloads.apache.org/spark/spark-4.0.0/spark-4.0.0-bin-hadoop3.tgz
!tar xf spark-4.0.0-bin-hadoop3.tgz

# get data for labs
!wget -nc -O around_the_world_in_80_days.txt https://www.gutenberg.org/ebooks/103.txt.utf-8

# install findspark
!pip install -q findspark

In [0]:
import os
import findspark

# set env vars for java and spark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-4.0.0-bin-hadoop3"

# start findspark so notebook can interact with spark
findspark.init()


In [0]:
# what does findspark do? use the ?? magic command to find out
# Note 1: in colab, this may open in a side panel
# Note 2: this magic command is often helpful when encountering an object in a
# notebook that is unfamiliar. More information will be displayed if it exists
?? findspark

# 1. Word Count

Instructions:  
For each cell marked "double-click and add explanation here" please answer the question in your own words.  
In the section where you complete the code to perform basic nlp text cleaning and exploration tasks, the goal is to chain all of the transformations together in a single function. For learning and exploration purposes, it is acceptable to have each step seperate, but the last cell in this section should be one function with all transformations chained together.  
For steps c and f, it is acceptable to use your favorite chatbot to generate a list of common stop words (c) and punctuation (e) for use in the code. As these are common steps in nlp/text processing tasks, there are pleanty of libraries to help with this such as nltk, but there is no need to import extra dependencies for this lab unless you are already familiar with working with them.

In [0]:
# start a spark session and create spark context for making rdd
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("word_count") \
    .getOrCreate()

sc = spark.sparkContext

In [0]:
# Defind the rdd
rdd = sc.textFile('/content/around_the_world_in_80_days.txt')

In [0]:
# view the first x lines of the rdd
rdd.take(20)

In [0]:
# example lambda function
words = rdd.flatMap(lambda lines: lines.split(' '))

In [0]:
# Note and explain the output of the below command
words

# Explaining the command

Writing a variable alone in a cell does the same as a print.

Here it displays the object type of the variable "words" wich is a RDD object and also where it comes from "PythonRDD.scala:56".


<ADD EXPLANATION HERE>

In [0]:
# Note and explain the output of the following command, focusing on the difference with the
# above command
words.collect()

# Explaining the command


The collect() function returns a list containing all the elements in the RDD words.

In [0]:
# nicer print
for w in words.collect():
    print(w)

In [0]:
# Print first x words
words.take(20)

In [0]:
# Use cell magic command to help understand what the rdd.flatMap function is doing in the next cell.
# Insert a text/markdown cell and explain in your own words.

?? rdd.flatMap

# rdd.flatMap

rdd.flatMap is a method that takes in argument a function and applies it to the Rdd passed in parameter while assigning it to a new flattened RDD. In the cell bellow it splits text lines into individual words, creates (word, 1) tuples for each word, and prints all the tuples.

In [0]:
# Initialize a word counter by creating a tuple with word and cound of 1
words = rdd.flatMap(lambda lines: lines.split(' ')) \
                    .map(lambda word: (word, 1))

for w in words.collect():
    print(w)

In [0]:
# a. count the occurence of each word

word_count= {}

for w in words.collect():
  if w[0] in word_count:
    word_count[w[0]] += 1
  else:
    word_count[w[0]] = 1

for w in word_count:
  print(w, word_count[w])

In [0]:
# b. a common first step in text analysis, change all capital letters to lower case
words_lower = {}
for word in word_count.keys():
  words_lower[word.lower()] = word_count[word]

for w in words_lower:
  print(w, words_lower[w])

In [0]:
# c. eliminate the stop words.

words_lower_stop_words = {}

stop_words = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and",
    "any", "are", "aren't", "as", "at", "be", "because", "been", "before", "being",
    "below", "between", "both", "but", "by", "can't", "cannot", "could", "couldn't",
    "did", "didn't", "do", "does", "doesn't", "doing", "don't", "down", "during",
    "each", "few", "for", "from", "further", "had", "hadn't", "has", "hasn't",
    "have", "haven't", "having", "he", "he'd", "he'll", "he's", "her", "here",
    "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i",
    "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "isn't", "it", "it's",
    "its", "itself", "let's", "me", "more", "most", "mustn't", "my", "myself",
    "no", "nor", "not", "of", "off", "on", "once", "only", "or", "other", "ought",
    "our", "ours", "ourselves", "out", "over", "own", "same", "shan't", "she",
    "she'd", "she'll", "she's", "should", "shouldn't", "so", "some", "such", "than",
    "that", "that's", "the", "their", "theirs", "them", "themselves", "then",
    "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've",
    "this", "those", "through", "to", "too", "under", "until", "up", "very", "was",
    "wasn't", "we", "we'd", "we'll", "we're", "we've", "were", "weren't", "what",
    "what's", "when", "when's", "where", "where's", "which", "while", "who",
    "who's", "whom", "why", "why's", "with", "won't", "would", "wouldn't", "you",
    "you'd", "you'll", "you're", "you've", "your", "yours", "yourself",
    "yourselves"
]


for w in words_lower:
  if  w not in stop_words:
    words_lower_stop_words[w] = words_lower[w]

for w in words_lower_stop_words:
  print(w, words_lower_stop_words[w])


In [0]:
# d. sort in alphabetical order

words_lower_stop_words_sorted = {}

for w in sorted(words_lower_stop_words):
  words_lower_stop_words_sorted[w] = words_lower_stop_words[w]

for w in words_lower_stop_words_sorted:
  print(w, words_lower_stop_words_sorted[w])


In [0]:
# e. sort descending by word frequency

words_lower_stop_words_sorted_freq = {}

for w in sorted(words_lower_stop_words_sorted, key=words_lower_stop_words_sorted.get, reverse=True):
  words_lower_stop_words_sorted_freq[w] = words_lower_stop_words_sorted[w]


for i, w in enumerate(words_lower_stop_words_sorted_freq ):
  print(w, words_lower_stop_words_sorted_freq[w])
  if i > 10:
    break

In [0]:
# f. remove punctuations and blank spaces

words_lower_stop_words_sorted_freq_punc: dict[str, int] = {}

punctuations = [
    "!", '"', "#", "$", "£", "€", '•', "%", "&", "'", "(", ")", "*", "+", ",", "-", ".", "/",
    ":", ";", "<", "=", ">", "?", "@", "[", "\\", "]", "^", "_", "`", "{", "|",
    "}", "~", " ", "“", "—", "”", "‘", "’"
]

for w, freq in words_lower_stop_words_sorted.items():
    cleaned = w
    for p in punctuations:
        cleaned = cleaned.replace(p, "")
    cleaned = cleaned.strip()
    if cleaned:
        words_lower_stop_words_sorted_freq_punc[cleaned] = freq

for i, (w, freq) in enumerate(words_lower_stop_words_sorted_freq_punc.items()):
    print(w, freq)



# 2. What does the following cell block do?
Comment the code below line by line after the provided hash-tag. You should be able to explain each line while respecting the pep8 style guide of 79 characters or less per line!

In [0]:
 # Create an RDD of tuples (name, age)
dataRDD = sc.parallelize([("Brooke", 20), ("Denny", 31), ("Jules", 30),
("TD", 35), ("Brooke", 25)])

# Try to undestand what this code does (line by line)
agesRDD = (dataRDD
  #Transform each record into (name,(age,1)), preparing for the aggregation
  .map(lambda x: (x[0], (x[1], 1)))
  #Groups by name and sums ages (x[0] + y[0]) and counts (x[1]+y[1])
  .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
  #Now we compute the average age per name by dividing total age by total count
  .map(lambda x: (x[0], x[1][0]/x[1][1])))

print(agesRDD.collect())



# Where to go from here

Further exploration for students who complete the lab before the end of the session or want to go further.

- perform eda on the original french version of the [book](https://www.gutenberg.org/ebooks/46541.txt.utf-8) and compare the two
- recomplete the exercises using a the docker install
- install java and spark directly onto host machine and either rexplore this notebook or perform eda on other data sets
- write a simple python timer function for seeing how quickly your rdd runs as written. change the order of the steps in order to make the rdd run as optimally as possible